<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn.

In [2]:
import numpy

def sigmoid(n):
    return 1 / (1 + numpy.exp(-n))
def logit(n):
    sig = sigmoid(n)
    return sig * (1 - sig)

In [337]:
import numpy
from typing import Optional


class FeedForwardLayer:
    def __init__(
            self,
            nodes: int,
            input_nodes: int,
            learning_rate: float = 1.0,
            weights: Optional[list] = None,
            activation_function: callable = sigmoid,
            activation_inverse: callable = logit
    ):
        """
        """

        self.nodes = nodes
        self.input_nodes = input_nodes
        self.learning_rate = learning_rate
        self.activation_function = activation_function
        self.activation_inverse = activation_inverse

        if weights is None:
            weights = numpy.random.random((input_nodes + 1, nodes))
        
        self.weights = weights


    def activate(
            self,
            X: numpy.ndarray
    ) -> numpy.ndarray:
        """
        """

        # self.input = X
        self.input = numpy.hstack((X, numpy.ones((X.shape[0], 1))))

        # print('activate: self.input')
        # print(self.input)

        layer_sum = numpy.dot(self.input, self.weights)
        self.output = self.activation_function(layer_sum)

        return (self.output)


    def descend_gradient(
            self,
            error: numpy.ndarray
    ) -> numpy.ndarray:
        """
        """

        # print('descend_gradient: self.input')
        # print(self.input)
        # print('descend_gradient: self.output')
        # print(self.output)
        # print('descend_gradient: error')
        # print(error)

        inv_out = self.activation_inverse(self.output)
        inp_T = self.input.T
        drop = False
        if inv_out.shape[1] < error.shape[1]:
            drop = True
            inv_out = numpy.hstack((inv_out, numpy.ones((inv_out.shape[0], 1))))

        # print('descend_gradient: inv_out')
        # print(inv_out)

        delta = error * inv_out

        # print('descend_gradient: delta')
        # print(delta)
        # print('descend_gradient: inp_T')
        # print(inp_T)
        # print('descend_gradient: numpy.dot(inp_T, delta)')
        # print(numpy.dot(inp_T, delta))

        if drop is True:
            # adjustments = adjustments[:,:-1]
            delta = delta[:,:-1]
        adjustments = numpy.dot(inp_T, delta)
        self.weights += adjustments * self.learning_rate

        parent_error = numpy.dot(delta, self.weights.T)
        return (parent_error)

    
    def __repr__(self) -> str:
        return (f'<FeedForwardLayer nodes={self.nodes} weights.shape={self.weights.shape}>')


In [338]:
numpy.array([[1],[2]]).shape

(2, 1)

In [339]:
from typing import List


class FeedForwardNeuralNetwork:
    def __init__(
            self,
            shape: List[int],
            n_iter: int = 100,
            learning_rate: float = 0.01
    ):
        self.n_iter = n_iter
        self.shape = shape
        self.learning_rate = learning_rate
        self.layers = None


    def initialize_nodes(self) -> None:
        self.layers = []
        input_nodes = self.shape[0]
        for nodes in self.shape[1:]:
            self.layers.append(
                FeedForwardLayer(nodes, input_nodes, learning_rate=self.learning_rate)
            )
            input_nodes = nodes


    def predict(self, X: numpy.ndarray) -> numpy.ndarray:
        layer_X = X
        for layer in self.layers:
            output = layer.activate(layer_X)
            layer_X = output

        return (output)

    
    def backward_propagate(self, error: numpy.ndarray) -> numpy.ndarray:
        # print('backward_propagate: error(initial)')
        # print(error)
        for layer in reversed(self.layers):
            # print('backward_propagate: layer')
            # print(layer)

            error = layer.descend_gradient(error)

            # print('backward_propagate: error')
            # print(error)

        return (error)


    def epoch(self, X: numpy.ndarray, y: numpy.ndarray) -> numpy.ndarray:
        output = self.predict(X)
        error = self.get_error(output, y)
        propagated_error = self.backward_propagate(error)
        
        return (error)


    def fit(
            self,
            X: numpy.ndarray,
            y: numpy.ndarray,
            n_iter: Optional[int] = None,
            refit: bool = True
    ) -> numpy.ndarray:
        if refit is True:
            self.initialize_nodes()
        
        if n_iter is None:
            n_iter = self.n_iter

        for i in range(n_iter):
            error = self.epoch(X, y)

        return (error)

    
    def __repr__(self) -> str:
        return (f'<FeedForwardNetwork shape={self.shape}>')


    @staticmethod
    def get_error(output: numpy.ndarray, y: numpy.ndarray) -> numpy.ndarray:
        return (y - output)



In [342]:
test = FeedForwardNeuralNetwork([2,4,2], learning_rate=1.0, n_iter=4000)
example_X = numpy.array([[.2,.4],[.4,.8]])
example_y = numpy.array([[.3,.3],[.6,.6]])

print(f'Final Error: {test.fit(example_X, example_y)}')
test.predict(example_X)


Final Error: [[-2.06639547e-07  2.00456457e-07]
 [ 1.88301990e-07 -1.82913529e-07]]


array([[0.30000021, 0.2999998 ],
       [0.59999981, 0.60000018]])

In [226]:
example_X[[0]]

array([[0.2, 0.4]])

In [243]:
for layer in test.layers:
    print(layer.weights)

[[0.99997829 0.14128624]
 [0.39608589 0.03669814]
 [0.65435081 0.71631048]]
[[0.82482298]
 [0.55798676]
 [0.30214996]]


In [182]:

test.predict(numpy.array([[.2, .3]]))

array([[0.19739318]])

In [345]:
X = numpy.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [0,1,0],
    [1,0,0],
    [1,1,1],
    [0,0,0],
])
y = numpy.array([[0],[1],[1],[1],[1],[0],[0]])
t2 = FeedForwardNeuralNetwork([3,4,1], learning_rate=0.5, n_iter=4000)
t2.fit(X, y)
t2.predict(X)

array([[0.05138094],
       [0.8784851 ],
       [0.87784219],
       [0.94136042],
       [0.94207784],
       [0.16334018],
       [0.08744767]])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row.

In [346]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [347]:
# input image dimensions
img_rows, img_cols = 28, 28

In [348]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 6s 1us/step


In [349]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [350]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

In [358]:
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [351]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [352]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [356]:
y_tr = y_train.reshape((-1,1))
y_tr

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [367]:
x_train[:1000].shape

(1000, 784)

In [370]:
y_tr[:100]

array([[0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],

In [371]:

t2 = FeedForwardNeuralNetwork([784,1], learning_rate=0.5, n_iter=4000)
t2.fit(x_train[:1000], y_tr[:1000])[:100]
t2.predict(x_train[:100])


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?